## Part 2: Optimization
    

Unkind people have spread rumours that Python is terribly slow. But that's not true. Pure Python trades convenience for performance, but we can always adjust the dial back -- do a little more work, and we can get excellent performance.

The trick is focusing this effort. Not eveything in your code needs to go fast. Hopefully you've now figured out where things are too slow, and we can now focus on optimising just those bits.

### Q: Why can Python be slow?

A: It's an interpreted language with dynamic typing.

In [ ]:
a = 0 

# Python doesn't compile, so can't optimise below 
# (we should be able to precalculate answer)
for i in range(100): 
    a += i # <- Will check types for 'a' and 'i' every time they are used (100 times)
    

Python can be slow because it doesn't make you define variable types (i.e. whether they are strings, integers, etc.). This is convenient, but means Python has to check the type of every variable each time it gets used.

Python is also an interpreted language, similar to R and Javascript, but unlike C++ or Java. That means the code doesn't get compiled before it runs (only simplified to byte code). Again, that's convenient (no waiting, and we can do things interactively), but means there's no compiler to try and optimise your code for you automatically, and there's an intermediate layer to interpret your code which slows things down a little.

But Python is 'batteries included' -- there are plenty of mature third-party packages that can reverse one or both of these characteristics for the bits of your code that need it.

## Options

* Restructurting your code
* NumPy - Work with arrays/matrices
* Pandas - Builds on NumPy for conveniently working with tables
* Cython - Converts bits of your code to low-level C code (which can be compiled)
* Numba & PyPy - Automatically compiles your code on the fly

### NumPy

In [ ]:
# Pure Python
from random import random

a = [random() for _ in range(100000)]
b = [random() for _ in range(100000)]

%timeit c = [x * y for x, y in zip(a, b)]

In [ ]:
# NumPy
import numpy as np

a = np.array(a)
b = np.array(b)

%timeit c = a * b

This should give about 40ms vs 40us -- that is NumPy is about 100x faster here!

Numpy includes a bunch of functions for working with arrays and matrics, including things like finding min/max, averages, and so on. 

If you're working with large sets of numbers, for instance doing linear algebra, then you should use NumPy rather than built-in Python types. This works a bit like MATLAB. You get to work with arrays and matrices, and there are many operations built-in, and they are all optimised to run fast. There are even more operations in the sister package SciPy.

### Pandas

In [ ]:
import pandas as pd
%matplotlib inline

raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'last_name': ['Miller', 'Jacobson', "Faye", 'Milner', 'Cooze'], 
        'age': [42, 52, 36, 24, 73], 
        'testScore': [25, 94, 57, 62, 72]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'testScore'])
df

In [ ]:
# A very small taste of what Pandas can do...

# Query
df[df['age'] > 50]

In [ ]:
# Do Math
df['age'].mean()

In [ ]:
# Import/export
df.to_csv('output.csv')

# Plot
df.plot.scatter('age', 'testScore')

I've just given a little taste here.

Pandas is a fantastic package for dealing with tabular data, think of it as a step up from NumPy (though it still uses NumPy under the covers). For example, a time-series of data points, or maybe a summary of results. 

It's fast, but also saves you from writing a heap of code. If nothing else, this is certainly a Python package you should learn... and we run dedicated training on it!

### Cython

In [ ]:
# Baseline -- Pure Python
def primes(kmax):
    result = []
    p = [None] * 1000
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

%timeit out = primes(10000)

In [ ]:
%%writefile cython_example.pyx

def primes(int kmax):
    cdef int n, k, i   # Let Cython know these are always 32 bit integers.
    cdef int p[1000]   # ...and to allocate an array of max length 1000.
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
%%writefile setup.py

from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("cython_example.pyx")
)

In [ ]:
%%bash
python setup.py build_ext --inplace 

In [ ]:
import cython_example
# dir(cython_example)
%timeit out = cython_example.primes(10000)

Cython lets you provide 'hints' so that it can figure out the types of your variables, and so convert it to C which can be compiled before it runs. Often this is really easy! 

Downside is you'll need to setup a C compiler (though you may already have one), and compile your code in advance.

In the case of this example, those couple little tweaks sped things up by ~40x!

### Numba

In [ ]:
import numba

@numba.jit
def primes(kmax):
    result = []
    p = [None] * 1000
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return np.array(result)

%timeit out = primes(10000)

With Numba, if you're lucky, you can just pop '@jit' at the top of your function and it will go fast! But in this case, it doesn't seem any faster!?

In the background, Numba tries to compile this code just before executing (unlike cython, where you do this in advance), and infer the types based on how you use it. If Numba can't figure out how to do this, it will just stick to using normal Python and your code will still work, but won't be compiled to speed it up.



In [ ]:
import numba

@numba.jit(nopython=True) # The 'nopython' part forces things to be compiled.
def primes(kmax):
    result = []
    p = [None] * 1000
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return np.array(result)

%timeit out = primes(10000)

If, however, you use `nopython=True`, Numba will insist that everything be compilable, and raise an error if it gets stuck. 


In [ ]:
import numba

@numba.jit(nopython=True)
def primes(kmax):
    result = []
    p = [np.nan] * 1000   # Use NumPy NaN instead of Pure Python 'None' so Numba can understand.
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return np.array(result)

%timeit out = primes(10000)

Only some Python constructs can be compiled and so you might need to make a few tweaks to your code, or provide hints on the signature of your function (that is, the datatype of its inputs and outputs).

In this case, the error message suggested that it didn't like our array of 'None'. Numba generally works seamlessly with NumPy, so let's create a list of Numpy NaN objects instead.

This now works, and executes about as fast as our Cython example, except Numba has figured out the datatypes for itelf.

Again, I'm just scratching the surface of Numba, this is again a very rich package for doing high performance work with Python.

### Don't overdo it...

<blockquote>We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil. ~Donald Knuth</blockquote>

<img src="efficiency.png" width="50%"/>

At this point, you've probably gathered that you could spend alot of time tweaking things, potentially more than you'll save!

Try not to go overboard. Your code doesn't need to be perfect, and indeed optimising everything can just make it difficult to understand and debug. You can always consider saving your time and using a bigger computer instead, which we'll talk about in the next part.

## Challenges!

Go here: http://go.unimelb.edu.au/dgh6

Select optimization challenge (1 to 3), and click 'Clone' to start your own instance.

You can login using your University email, or create a (free) Microsoft account.

**Challenge 1**
* The second approach is faster because we are allocating all the memory we need in one step, rather than having to create a new (slightly bigger) one over and over again.
* ...so try to allocate all the space you need for your computations in one go (you can always use NaN to mark empty data points).

**Challenge 2**
* Using the built-in NumPy methods is vastly quicker than doing it yourself! 
* This is because there are many low-level highly-optimised libraries available for doing these sorts of calculations, which NumPy gives you an interface to

**Challenge 3**
* Can use:  ```data[data['age'] < 30]['name'].values```
* For this (very small) dataset, it isn't any faster. But for bigger datasets it certainly is. 
* It's more readable.